In [ ]:
!pip install pywapor --quiet

#### Running pyWaPOR at level-3

We've already explored how to run pyWaPOR at level-1 and looked at what differences there are between the different levels with regards to their input datasets. Here we're going to run pyWaPOR at level-3.

In [ ]:
folder = r"/Users/hmcoerver/Local/5_level3"
timelim = ["2021-08-11", "2021-08-21"]
lonlim = [68.5, 69.0]
latlim = [25.5, 26.0]

sources = "level_3"

> **Question**
> 
> We also need import the pywapor package, remember how to do that?

In [ ]:
# Import the pywapor package here.

Now that we have defined the inputs to `pywapor.pre_et_look.main` and imported the pywapor package, we can run the function.

> **Question**
>
> Run `pre_et_look` (and don't forget to pass the keyword-argument to select level-3 instead of level-1 too)!

In [ ]:
# Run pre_et_look here

Now that we have prepared our inputs, let's see if everything is as expected.

> **Question**
>
> * Can you check the resolution of the data, how does it compare to the level-1 data we ran yesterday?
> 
> * Try plotting some of the variables, do you see anything unusual?

In [ ]:
# Check the resolution of the ds here.

In [ ]:
# Create some plots here.

We could now use this dataset to run the model, but let's first try to improve some things first.

#### Gapfilling

On May 31, 2003, the Scan Line Corrector (SLC), which compensates for the forward motion of the Landsat-7 satellite, failed. As a result, Landsat-7 images taken after this date have lines of missing data.

We can see the problem more clearly by looking at the Landsat-7 images individually. `pre_et_look` has saved the individual landsat-7 scenes in a file called `"LE07_SR.nc"`. To open it, we need pass the full path to the file to the function called `pywapor.general.processing_functions.open_ds`. 

In [ ]:
import os
ls7_path = os.path.join(folder, "LANDSAT", "LE07_SR.nc")
ls7_ds = pywapor.general.processing_functions.open_ds(ls7_path)

> **Question**
>
> * How many scenes do you find in this dataset?
>
> * How many of the scenes in this dataset were used to create the NDVI composites in the dataset above (i.e. in `ds`)?

In [ ]:
# Inspect the ls7_ds dataset here

Let's create a plot of one Landsat-7 scene.

In [ ]:
ls7_ds.ndvi.isel(time=1).plot()

Here the scanline problem is clearly visible. In the previous excercise we experimented with adjusted the sources dictionary to change the spatial interpolation used for air temperature and we toggled the lapse rate correction on and off.

It is also possible to pass enhancer functions (such as lapse_rate) to specific variables from specific sources. Here we are going to try to use a gap filling function on the albedo and ndvi data coming from Landsat-7.

Let's start by getting the information dictionary for level-3.

In [ ]:
lvl3_info = pywapor.general.levels.pre_et_look_levels("level_3")

We can look at the information regarding NDVI like this.

In [ ]:
lvl3_info["ndvi"]

> **Question**
>
> What is the current value for the enhancers of Landsat-7?

We can check what `"default"` means exactly, by running this function.

In [ ]:
default_enhancers = pywapor.collect.product.LANDSAT.default_post_processors('LE07_SR', ["ndvi"])["ndvi"]
default_enhancers

In order to apply fill the empty scanlines, we can add a gap filling function to this list and replace the `"default"` string with the list.

Let's start by importing a gap filling function.

In [ ]:
from pywapor.enhancers.gap_fill import gap_fill

> **Question**
> 
> Do you know how to open the documentation for this function?

In [ ]:
# open the documentation for gap_fill

We'll need to include this function in the lvl3_info for ndvi.

First we'll add the function to the default enhancers.

In [ ]:
all_enhancers = default_enhancers + [gap_fill]

Then we put the list into the lvl3_info dictionary.

In [ ]:
lvl3_info["ndvi"]["products"][1]["enhancers"] = all_enhancers

Before running `pre_et_look` again, we'll delete the Landsat-7 file we inspected earlier, in order to make sure that it really get regenerated with the gap-filling.

In [ ]:
os.remove(ls7_path)

Then we can finally run `pre_et_look` again!

In [ ]:
ds_filled  = pywapor.pre_et_look.main(folder, latlim, lonlim, timelim, sources = lvl3_info)

> **Question**
> 
> * Do you see any trace of the gap filling in the log?
>
> * Create a new plot of the ndvi, does it look better?

In [ ]:
# Plot an ndvi map here.

#### Your turn!

> **Question**
>
> * Did we apply the gap filling to the albedo (r0) too?
>
> * Can you do it? 
>
> * Then run the `et_look` model.